In [ ]:
import os
import pyspark
import findspark
findspark.init() 
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [ ]:
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))
print("PATH:", os.environ.get("PATH"))

from pyspark import SparkContext
try:
    sc = SparkContext.getOrCreate()
    print("SparkContext initialized successfully!")
except Exception as e:
    print(f"Error initializing SparkContext: {e}")

JAVA_HOME: C:\Program Files\Java\jdk-17
PATH: c:\Users\joaqu\bigdata310\Scripts;C:\Users\joaqu\bigdata310\Scripts;C:\Program Files\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Common Files\Oracle\Java\java8path;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files\dotnet\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\Microsoft\Azure Functions Core Tools\;C:\Users\joaqu\AppData\Local\Programs\Python\Python312;C:\Users\joaqu\AppData\Local\Programs\Python\Python312\Scripts;C:\Program Files\Docker\Docker\resources\bin;C:\Spark\bin;C:\Spark\bin;C:\Program Files\Java\jdk-17\bin;C:\Spark\bin;C:\Users\joaqu\AppData\Local\Programs\Python\Python310;C:\Program Files\Git\cmd;C:\Users\joa

In [3]:
from pyspark.sql.dataframe import DataFrame
from typing import Dict, List

def read_from_db(spark, config: Dict[str, str], query: str) -> DataFrame:
    """
    Reads data from an Oracle database into a Spark DataFrame.

    Args:
        spark (SparkSession): Pre-created SparkSession object.
        config (Dict[str, str]): Database configuration dictionary with keys `db_user`, `db_password`, and `db_dsn`.
        query (str): SQL query to execute.

    Returns:
        DataFrame: Spark DataFrame containing the query result.
    """
    url = f"jdbc:oracle:thin:@{config['db_dsn']}?ssl=true"

    
    return spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", f"({query})") \
        .option("user", config["db_user"]) \
        .option("password", config["db_password"]) \
        .option("driver", "oracle.jdbc.OracleDriver") \
        .load()

In [4]:
import yaml

def load_config(file_path):
    """
    Loads a YAML configuration file and returns the contents as a dictionary.
    
    :param file_path: Path to the YAML file.
    :return: Dictionary with configuration values.
    """
    try:
        with open(file_path, 'r') as file:
            config = yaml.safe_load(file)
            return config
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except yaml.YAMLError as e:
        print(f"Error parsing YAML file: {e}")
        return None


In [13]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read Utility Matrix") \
    .master("local[*]") \
    .config("spark.driver.memory", "2G") \
    .config("spark.jars", "C:/Spark/jars/ojdbc17.jar") \
    .getOrCreate()

# Load database configuration
config = load_config(r"C:\Users\joaqu\OneDrive\Documentos\Trento\Data_Mining\Data-Mining-Project\src\config.yaml")

# Read utility matrix from the database
print("Verifying utility matrix in the database...")
try:
    result_df = read_from_db(spark, config, "SELECT * FROM utility_matrix_user_item")
    result_df.show(10, truncate=False)
except Exception as e:
    print(f"Error reading from the database: {e}")

# Optional: Convert to Pandas for further analysis
pandas_df = result_df.toPandas()
print(pandas_df.head())


FileNotFoundError: [WinError 2] The system cannot find the file specified